In [1]:
import os
os.environ['MANTLE_TARGET'] = 'ice40'

import magma as m

Creating a Counter.

![](./images/counter_diagram.jpg)

We will use the builder pattern to define a counter module.

`DefineCircuit` takes as a first argument the name of the circuit and then an argument list. The argument list is a collection of ports. The argyment list hasan even number of arguments of the form `port1_name, port1_type, port2_name, port2_type, ...`. The argument list is created programmatically, and then passed to `DefineCircuit`. using the python `*` operator.

For this example we compose a list of named arguments combined with the output of the `ClockInterface` function.  `ClockInterface`  produces a standard set of inputs for a clock, such as clock enable and/or reset. In this case, we only include a clock (by default) and an optional clock enable because `has_ce=has_ce`.

We instance an n-bit `Adders` and `Register` from the `mantle` library. We wire them up based on our counter diagram above. Finally, we call `m.wireclock` to automatically wire up the clock inputs to the `Register` instance.

In [2]:
from mantle import Add, Register

def DefineCounter(n, has_ce=False):
    
    class _Counter(m.Circuit):
        name = "Counter{}{}".format(n, "_hasce" if has_ce else "")
        IO = ["O", m.Out(m.Bits(n)), "COUT", m.Out(m.Bit)] + m.ClockInterface(has_ce=has_ce)
        
        @classmethod
        def definition(io):
            add = Add(n, cin=False, cout=True)
            reg = Register(n, has_ce=has_ce)

            m.wire( reg.O, add.I0 )
            m.wire( m.bits(1, n), add.I1 )

            m.wire(add.O, reg.I)

            m.wire( reg.O, io.O )

            m.wire( add.COUT, io.COUT )

            m.wireclock(io, reg)
    
            m.wiredefaultclock(io, reg)

    return _Counter

import mantle lattice ice40
import mantle lattice mantle40


In [3]:
from loam.boards.icestick import IceStick

icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()
icestick.D5.on()

main = icestick.main()

counter4 = DefineCounter(5, has_ce=True)()
counter23 = DefineCounter(23)()
m.wire(counter23.COUT, counter4.CE)
m.wire(counter4.O, m.bits([main.D1, main.D2, main.D3, main.D4, main.D5]))

m.compile("build/counter_builder", main)

compiling FullAdder
compiling Add5Cout
compiling Register5CE
compiling Counter5_hasce
compiling Add23Cout
compiling Register23
compiling Counter23
compiling main


Compile and flash our design onto the icestick. The leds should blink in a 5-bit counter pattern. Change the size of the 23-bit control counter or replace it with a different circuit to experiment with the counting pattern.

In [4]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif counter_builder.blif' counter_builder.v
arachne-pnr -q -d 1k -o counter_builder.txt -p counter_builder.pcf counter_builder.blif
icepack counter_builder.txt counter_builder.bin

#!iceprog counter_builder.bin

/Users/hanrahan/git/magma/notebooks/icestick/build
